In [54]:
import os
import glob
import pandas as pd
import numpy as np
import cv2
import locale
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from sklearn.model_selection import train_test_split

import plotly.express as px
import plotly.graph_objects as go

In [55]:
dataset_path = "../data/raw/houses-dataset/houses_dataset/"

In [56]:
cols = [
    "bedrooms",
    "bathrooms",
    "area",
    "zipcode",
    "price"
]
df = pd.read_csv(dataset_path + "HousesInfo.txt", sep=" ", header=None, names=cols)
df.head()

,bedrooms,bathrooms,area,zipcode,price
0,4,4.0,4053,85255,869500
1,4,3.0,3343,36372,865200
2,3,4.0,3923,85266,889000
3,5,5.0,4022,85262,910000
4,3,4.0,4116,85266,971226


In [57]:
# zipcode to categorical
df["zipcode"] = df["zipcode"].astype("str")

In [58]:
zipcodes, count = np.unique(
    df["zipcode"],
    return_counts=True
)
# dict(zip(zipcodes, count))
df.shape

(535, 5)

In [59]:
# plot histogram of zip codes
fig = px.histogram(
    df,
    x="zipcode",
    nbins=70,
    title="Histogram of Zip Codes"
)
fig.show()

In [60]:
# delete zip codes with less than 25 houses
for (zipcode, count) in zip(zipcodes, count):
    if count < 25:
        idxs = df[df["zipcode"] == zipcode].index
        df.drop(idxs, inplace=True)
df.shape

(362, 5)

In [61]:
# plot histogram of zip codes
fig = px.histogram(
    df,
    x="zipcode",
    nbins=70,
    title="Histogram of Zip Codes"
)
fig.show()

In [62]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop("price", axis=1),
    df["price"],
    test_size=0.25,
    random_state=42
)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((271, 4), (91, 4), (271,), (91,))

In [63]:
# Normalization
max_price = df["price"].max()
y_train = y_train / max_price
y_test = y_test / max_price

In [64]:
# Normalization
scaler = MinMaxScaler()
continues_cols = ["bedrooms", "bathrooms", "area"]
x_train_continues = scaler.fit_transform(x_train[continues_cols])
x_test_continues = scaler.transform(x_test[continues_cols])

In [65]:
# one hot encoding of zip codes
lb = LabelBinarizer().fit(df["zipcode"])
x_train_discrete = lb.transform(x_train["zipcode"])
x_test_discrete = lb.transform(x_test["zipcode"])

In [66]:
x_train = np.hstack([x_train_continues, x_train_discrete])
x_test = np.hstack([x_test_continues, x_test_discrete])

print(x_train.shape, x_test.shape)

(271, 10) (91, 10)


In [67]:
model = Sequential()
model.add(Dense(8, input_shape=(x_train.shape[1],), activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation="linear"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 88        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________


2023-01-28 21:03:38.270119: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/smbh/venvs/house_price_prediction/lib/python3.10/site-packages/cv2/../../lib64:
2023-01-28 21:03:38.270369: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-28 21:03:38.270384: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hope): /proc/driver/nvidia/version does not exist
2023-01-28 21:03:38.271030: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild Ten

In [69]:
opt = Adam(learning_rate=1e-3, weight_decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [71]:
history = model.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=200,
    batch_size=16
)

Epoch 1/200
17/17 [==============================] - 0s 3ms/step - loss: 22.0274 - val_loss: 25.2471
Epoch 2/200
17/17 [==============================] - 0s 2ms/step - loss: 20.7101 - val_loss: 25.0041
Epoch 3/200
17/17 [==============================] - 0s 2ms/step - loss: 21.5492 - val_loss: 24.2439
Epoch 4/200
17/17 [==============================] - 0s 2ms/step - loss: 20.8971 - val_loss: 25.8179
Epoch 5/200
17/17 [==============================] - 0s 2ms/step - loss: 20.6416 - val_loss: 24.2647
Epoch 6/200
17/17 [==============================] - 0s 3ms/step - loss: 20.9103 - val_loss: 25.7382
Epoch 7/200
17/17 [==============================] - 0s 2ms/step - loss: 22.4227 - val_loss: 26.6255
Epoch 8/200
17/17 [==============================] - 0s 2ms/step - loss: 22.8008 - val_loss: 26.9771
Epoch 9/200
17/17 [==============================] - 0s 2ms/step - loss: 22.9308 - val_loss: 24.7052
Epoch 10/200
17/17 [==============================] - 0s 2ms/step - loss: 20.9637 - val_los

In [73]:
# Plot training and validation loss
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=history.history["loss"],
    name="Training Loss"
))
fig.add_trace(go.Scatter(
    y=history.history["val_loss"],
    name="Validation Loss"
))
fig.update_layout(
    title="Training and Validation Loss",
    xaxis_title="Epoch",
    yaxis_title="Loss"
)
fig.show()

In [75]:
preds = model.predict(x_test)

3/3 [==============================] - 0s 1ms/step


In [76]:
diff = preds.flatten() - y_test
percent_diff = (diff / y_test) * 100
abs_percent_diff = np.abs(percent_diff)

mean = np.mean(abs_percent_diff)
std = np.std(abs_percent_diff)

In [79]:
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print(f"avg. house price: {locale.currency(df['price'].mean(), grouping=True)}")
print(f"mean: {mean:.2f}%, std: {std:.2f}%")

avg. house price: $533,388.27
mean: 24.48%, std: 21.28%
